In [1]:
# train_val_cls.py

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import math
import random
import shutil
import argparse
import importlib
import numpy as np
import tensorflow as tf
from datetime import datetime

/root/anaconda3/envs/tf1.4_gpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/root/anaconda3/envs/tf1.4_gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import sys  
sys.path.append('../')

In [3]:
import pointfly as pf

In [4]:
# path for training data
path = '../../data/modelnet/train_files.txt'
# path for validation data
path_val = '../../data/modelnet/test_files.txt'
# Path to folder for saving check points and summary
save_folder = '../../models/cls'
# model to use
model = 'pointcnn_cls'
# Setting to use
setting = 'modelnet_x2_l4'

In [5]:
arg_model = 'pointcnn_cls'

In [6]:
time_string = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
root_folder = os.path.join(save_folder, '%s_%s_%d_%s' % (model, setting, os.getpid(), time_string))

In [7]:
root_folder

'../../models/cls/pointcnn_cls_modelnet_x2_l4_26762_2018-02-06-02-57-05'

In [8]:
if not os.path.exists(root_folder):
    os.makedirs(root_folder)

In [9]:
#sys.stdout = open(os.path.join(root_folder, 'log.txt'), 'w')

In [10]:
print('PID:', os.getpid())

PID: 26762


In [11]:
# import pointcnn_cls

model = importlib.import_module(model)

In [12]:
type(model)

module

In [12]:
#os.path.dirname(__file__)

NameError: name '__file__' is not defined

In [13]:
setting_path = os.path.join('../', arg_model)

In [14]:
setting_path

'../pointcnn_cls'

In [15]:
sys.path.append(setting_path)
setting = importlib.import_module(setting)

In [16]:
type(setting)

module

In [17]:
num_epochs = setting.num_epochs
batch_size = setting.batch_size
sample_num = setting.sample_num
step_val = setting.step_val
num_class = setting.num_class
rotation_range = setting.rotation_range
rotation_range_val = setting.rotation_range_val
jitter = setting.jitter
jitter_val = setting.jitter_val

In [18]:
print('num_epochs:', num_epochs)
print('batch_size:', batch_size)
print('sample_num:', sample_num)
print('step_val:', step_val)
print('num_class:', num_class)
print('rotation_range:', rotation_range)
print('rotation_range_val:', rotation_range_val)
print('jitter:', jitter)
print('jitter_val:', jitter_val)

num_epochs: 2048
batch_size: 200
sample_num: 1024
step_val: 500
num_class: 40
rotation_range: [0, 3.141592653589793, 0, 'u']
rotation_range_val: [0, 0, 0, 'u']
jitter: 0.0
jitter_val: 0.0


In [ ]:
# Prepare inputs

In [19]:
print('{}-Preparing datasets...'.format(datetime.now()))
data_train, label_train, data_val, label_val = setting.load_fn(path, path_val)

2018-02-06 02:57:35.076008-Preparing datasets...


In [20]:
data_train.shape, label_train.shape, data_val.shape, label_val.shape

((9840, 2048, 6), (9840,), (2468, 2048, 6), (2468,))

In [21]:
data_train[0][0]

array([-0.32291198,  0.21188144,  0.31092408, -0.32291198,  0.21188144,
        0.31092408], dtype=float32)

In [22]:
# Save ply file
folder = os.path.join(root_folder, 'pts')
print('{}-Saving samples as .ply files to {}...'.format(datetime.now(), folder))

2018-02-06 02:58:00.202182-Saving samples as .ply files to ../../models/cls/pointcnn_cls_modelnet_x2_l4_26762_2018-02-06-02-57-05/pts...


In [23]:
# the number of sample  points to be saved in ply file
sample_num_for_ply = min(512, data_train.shape[0])

In [24]:
setting.map_fn

In [25]:
if setting.map_fn is None:
    data_sample = data_train[:sample_num_for_ply]

In [26]:
data_sample.shape

(512, 2048, 6)

In [27]:
setting.save_ply_fn(data_sample, folder)

ValueError: could not assign tuple of length 6 to structure with 3 fields.

In [ ]:
if setting.save_ply_fn is not None:
    folder = os.path.join(root_folder, 'pts')
    print('{}-Saving samples as .ply files to {}...'.format(datetime.now(), folder))
    sample_num_for_ply = min(512, data_train.shape[0])
    if setting.map_fn is None:
        data_sample = data_train[:sample_num_for_ply]
    else:
        data_sample_list = []
        for idx in range(sample_num_for_ply):
            data_sample_list.append(setting.map_fn(data_train[idx], 0)[0])
        data_sample = np.stack(data_sample_list)
    setting.save_ply_fn(data_sample, folder)

In [28]:
num_train = data_train.shape[0]
point_num = data_train.shape[1]
num_val = data_val.shape[0]
print('{}-{:d}/{:d} training/validation samples.'.format(datetime.now(), num_train, num_val))

2018-02-06 02:59:24.216375-9840/2468 training/validation samples.


In [29]:
######################################################################
# Placeholders
indices = tf.placeholder(tf.int32, shape=(None, None, 2), name="indices")
xforms = tf.placeholder(tf.float32, shape=(None, 3, 3), name="xforms")
rotations = tf.placeholder(tf.float32, shape=(None, 3, 3), name="rotations")
jitter_range = tf.placeholder(tf.float32, shape=(1), name="jitter_range")
global_step = tf.Variable(0, trainable=False, name='global_step')
is_training = tf.placeholder(tf.bool, name='is_training')

data_train_placeholder = tf.placeholder(data_train.dtype, data_train.shape)
label_train_placeholder = tf.placeholder(label_train.dtype, label_train.shape)
data_val_placeholder = tf.placeholder(data_val.dtype, data_val.shape)
label_val_placeholder = tf.placeholder(label_val.dtype, label_val.shape)
handle = tf.placeholder(tf.string, shape=[])

In [31]:
data_train_placeholder

<tf.Tensor 'Placeholder:0' shape=(9840, 2048, 6) dtype=float32>

In [32]:
data_train_placeholder.shape

TensorShape([Dimension(9840), Dimension(2048), Dimension(6)])

In [36]:
# create training datset from data and label placeholder
dataset_train = tf.data.Dataset.from_tensor_slices((data_train_placeholder, label_train_placeholder))

In [37]:
??tf.data.Dataset.from_tensor_slices

In [46]:
type(dataset_train)

tensorflow.python.data.ops.dataset_ops.ShuffleDataset

In [ ]:
'''
if setting.map_fn is not None:
    dataset_train = dataset_train.map(lambda data, label: tuple(tf.py_func(
        setting.map_fn, [data, label], [tf.float32, label.dtype])), num_parallel_calls=setting.num_parallel_calls)
'''

In [39]:
batch_size

200

In [40]:
??dataset_train.shuffle

In [41]:
dataset_train = dataset_train.shuffle(buffer_size=batch_size * 4)

In [42]:
setting.keep_remainder

True

In [45]:
# before dataset_train.batch
dataset_train

<ShuffleDataset shapes: ((2048, 6), ()), types: (tf.float32, tf.int32)>

In [47]:
# keep the remainder part of the dataset
if setting.keep_remainder:
    dataset_train = dataset_train.batch(batch_size)
    batch_num_per_epoch = math.ceil(num_train / batch_size)
else:
    dataset_train = dataset_train.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
    batch_num_per_epoch = math.floor(num_train / batch_size)

In [48]:
# after dataset_train.batch
dataset_train

<BatchDataset shapes: ((?, 2048, 6), (?,)), types: (tf.float32, tf.int32)>

In [49]:
# how many batches to perform in each epoch
batch_num_per_epoch

50

In [53]:
# total batch_num from all num_epochs = total iters.
batch_num = batch_num_per_epoch * num_epochs
batch_num

102400

In [58]:
print('{}-{:d} training batches.'.format(datetime.now(), batch_num))

2018-02-06 04:39:56.352781-102400 training batches.


In [61]:
# after dataset_train.repeat()
dataset_train = dataset_train.repeat()
dataset_train

<RepeatDataset shapes: ((?, 2048, 6), (?,)), types: (tf.float32, tf.int32)>

In [62]:
# after make_initializable_iterator(), explcitly initialize the iterator
iterator_train = dataset_train.make_initializable_iterator()
iterator_train

In [63]:
dataset_val = tf.data.Dataset.from_tensor_slices((data_val_placeholder, label_val_placeholder))

In [66]:
setting.map_fn is not None

False

In [67]:
if setting.map_fn is not None:
    dataset_val = dataset_val.map(lambda data, label: tuple(tf.py_func(
        setting.map_fn, [data, label], [tf.float32, label.dtype])), num_parallel_calls=setting.num_parallel_calls)

In [68]:
if setting.keep_remainder:
    dataset_val = dataset_val.batch(batch_size)
    batch_num_val = math.ceil(num_val / batch_size)
else:
    dataset_val = dataset_val.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
    batch_num_val = math.floor(num_val / batch_size)

In [72]:
iterator_val = dataset_val.make_initializable_iterator()

In [73]:
iterator = tf.data.Iterator.from_string_handle(handle, dataset_train.output_types, dataset_train.output_shapes)

In [77]:
(pts_fts, labels) = iterator.get_next()

In [78]:
type(pts_fts)

tensorflow.python.framework.ops.Tensor

In [79]:
pts_fts.shape

TensorShape([Dimension(None), Dimension(2048), Dimension(6)])

In [80]:
type(labels)

tensorflow.python.framework.ops.Tensor

In [81]:
labels.shape

TensorShape([Dimension(None)])

In [82]:
features_augmented = None

In [83]:
setting.data_dim

6

In [84]:
setting.jitter_val

0.0

In [85]:
setting.with_normal_feature

AttributeError: module 'modelnet_x2_l4' has no attribute 'with_normal_feature'

In [86]:
setting.use_extra_features

False

In [ ]:
# Here setting.use_extra_features ==False, just perform tf.split on pts_fts

In [87]:
if setting.data_dim > 3:
    points, features = tf.split(pts_fts, [3, setting.data_dim - 3], axis=-1, name='split_points_features')
    if setting.use_extra_features:
        features_sampled = tf.gather_nd(features, indices=indices, name='features_sampled')
        if setting.with_normal_feature:
            features_augmented = pf.augment(features_sampled, rotations)
        else:
            features_augmented = features_sampled
else:
    points = pts_fts

In [88]:
points

<tf.Tensor 'split_points_features:0' shape=(?, 2048, 3) dtype=float32>

In [89]:
features

<tf.Tensor 'split_points_features:1' shape=(?, 2048, 3) dtype=float32>

In [93]:
# augment the points (sampled points)
points_sampled = tf.gather_nd(points, indices=indices, name='points_sampled')
points_augmented = pf.augment(points_sampled, xforms, jitter_range)

In [94]:
num_class

40

In [95]:
net = model.Net(points=points_augmented, features=features_augmented, num_class=num_class,
                is_training=is_training, setting=setting)

In [99]:
logits, probs = net.logits, net.probs

In [101]:
logits

<tf.Tensor 'cond/Merge:0' shape=(?, ?, 40) dtype=float32>

In [105]:
probs

<tf.Tensor 'probs:0' shape=(?, ?, 40) dtype=float32>

In [109]:
tf.shape(probs)[0]

<tf.Tensor 'strided_slice_13:0' shape=() dtype=int32>

In [110]:
tf.shape(probs)[1]

<tf.Tensor 'strided_slice_14:0' shape=() dtype=int32>

In [114]:
tf.shape(probs)[2]

<tf.Tensor 'strided_slice_18:0' shape=() dtype=int32>

In [102]:
labels

<tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>

In [103]:
labels_2d = tf.expand_dims(labels, axis=-1, name='labels_2d')

In [116]:
labels_tile = tf.tile(labels_2d, (1, tf.shape(probs)[1]), name='labels_tile')

In [ ]:
# expand Inserts a dimension of 1 at the end of labels
labels_2d = tf.expand_dims(labels, axis=-1, name='labels_2d')
# replicating labels_2d by (1, ) multiples
labels_tile = tf.tile(labels_2d, (1, tf.shape(probs)[1]), name='labels_tile')

In [117]:
loss_op = tf.losses.sparse_softmax_cross_entropy(labels=labels_tile, logits=logits)
t_1_acc_op = pf.top_1_accuracy(probs, labels_tile)

In [118]:
_ = tf.summary.scalar('loss/train', tensor=loss_op, collections=['train'])
_ = tf.summary.scalar('t_1_acc/train', tensor=t_1_acc_op, collections=['train'])

loss_val_avg = tf.placeholder(tf.float32)
t_1_acc_val_avg = tf.placeholder(tf.float32)
_ = tf.summary.scalar('loss/val', tensor=loss_val_avg, collections=['val'])
_ = tf.summary.scalar('t_1_acc/val', tensor=t_1_acc_val_avg, collections=['val'])


In [119]:
setting.learning_rate_base

0.01

In [120]:
global_step

<tf.Variable 'global_step:0' shape=() dtype=int32_ref>

In [121]:
setting.decay_steps

8000

In [122]:
setting.decay_rate

0.5

In [130]:
setting.weight_decay

0.0

In [123]:
??tf.train.exponential_decay

In [124]:
lr_exp_op = tf.train.exponential_decay(setting.learning_rate_base, global_step, setting.decay_steps,
                                       setting.decay_rate, staircase=True)

In [125]:
lr_exp_op

<tf.Tensor 'ExponentialDecay:0' shape=() dtype=float32>

In [126]:
setting.learning_rate_min

1e-06

In [127]:
lr_clip_op = tf.maximum(lr_exp_op, setting.learning_rate_min)

In [128]:
lr_clip_op

<tf.Tensor 'Maximum:0' shape=() dtype=float32>

In [131]:
reg_loss = setting.weight_decay * tf.losses.get_regularization_loss()

In [132]:
reg_loss

<tf.Tensor 'mul_16:0' shape=() dtype=float32>

In [133]:
setting.optimizer

'adam'

In [134]:
setting.epsilon

0.01

In [135]:
if setting.optimizer == 'adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=lr_clip_op, epsilon=setting.epsilon)
elif setting.optimizer == 'momentum':
    optimizer = tf.train.MomentumOptimizer(learning_rate=lr_clip_op, momentum=0.9, use_nesterov=True)

In [137]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [138]:
update_ops

[<tf.Tensor 'xconv_1_nn_pts_local_bn/AssignMovingAvg:0' shape=(3,) dtype=float32_ref>,
 <tf.Tensor 'xconv_1_nn_pts_local_bn/AssignMovingAvg_1:0' shape=(3,) dtype=float32_ref>,
 <tf.Tensor 'xconv_1_fts_bn/AssignMovingAvg:0' shape=(32,) dtype=float32_ref>,
 <tf.Tensor 'xconv_1_fts_bn/AssignMovingAvg_1:0' shape=(32,) dtype=float32_ref>,
 <tf.Tensor 'xconv_2_nn_pts_local_bn/AssignMovingAvg:0' shape=(3,) dtype=float32_ref>,
 <tf.Tensor 'xconv_2_nn_pts_local_bn/AssignMovingAvg_1:0' shape=(3,) dtype=float32_ref>,
 <tf.Tensor 'xconv_2_fts_bn/AssignMovingAvg:0' shape=(64,) dtype=float32_ref>,
 <tf.Tensor 'xconv_2_fts_bn/AssignMovingAvg_1:0' shape=(64,) dtype=float32_ref>,
 <tf.Tensor 'xconv_3_nn_pts_local_bn/AssignMovingAvg:0' shape=(3,) dtype=float32_ref>,
 <tf.Tensor 'xconv_3_nn_pts_local_bn/AssignMovingAvg_1:0' shape=(3,) dtype=float32_ref>,
 <tf.Tensor 'xconv_3_fts_bn/AssignMovingAvg:0' shape=(128,) dtype=float32_ref>,
 <tf.Tensor 'xconv_3_fts_bn/AssignMovingAvg_1:0' shape=(128,) dtype=floa

In [141]:
type([None])

list

In [142]:
xconv_params

NameError: name 'xconv_params' is not defined

In [143]:
x = 2

# K, D, P, C
xconv_params = [(8, 1, -1, 16 * x),
                (12, 2, 384, 32 * x),
                (16, 2, 128, 64 * x),
                (16, 3, 128, 128 * x)]

In [144]:
xconv_params

[(8, 1, -1, 32), (12, 2, 384, 64), (16, 2, 128, 128), (16, 3, 128, 256)]

In [145]:
xconv_params[0][-1]

32

In [147]:
for layer_idx, layer_param in enumerate(xconv_params):
    print(str(layer_idx)+" : "+str(layer_param))

0 : (8, 1, -1, 32)
1 : (12, 2, 384, 64)
2 : (16, 2, 128, 128)
3 : (16, 3, 128, 256)


In [148]:
points_augmented.shape

TensorShape([Dimension(None), Dimension(None), Dimension(3)])

In [150]:
fc_params = [(128 * x, 0.0), (64 * x, 0.5)]

In [151]:
fc_params

[(256, 0.0), (128, 0.5)]

In [152]:
for layer_idx, layer_param in enumerate(fc_params):
    print(str(layer_idx)+" : "+str(layer_param))

0 : (256, 0.0)
1 : (128, 0.5)


In [153]:
weight_decay

NameError: name 'weight_decay' is not defined

In [154]:
global_step

<tf.Variable 'global_step:0' shape=() dtype=int32_ref>

In [155]:
tf.data.Dataset.range(100)

<RangeDataset shapes: (), types: tf.int64>

In [157]:
A = np.array([[1,2],
         [3,4]])

In [159]:
A[::1,1]

array([2, 4])

In [160]:
A[1,1]

4

In [161]:
A[:1,1]

array([2])